# Exploring Lab preprocessing

- Align to unified equivalent label
- restrict to selected variables
- convert to target units of measure
- restrict to plausible range
- log descriptive stats

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import numpy as np

In [ ]:
lab_df_path = '/Users/jk1/temp/mimic/extraction/lab_df.csv'

In [ ]:
verbose = True

In [ ]:
lab_df = pd.read_csv(lab_df_path)

In [ ]:
selected_reference_values = pd.read_excel('./selected_lab_values.xlsx')

In [ ]:
lab_df.head()

In [ ]:
lab_df.label.unique()

In [ ]:
selected_reference_values

## Align to unified equivalent labels

In [ ]:
aligned_lab_df = lab_df.copy()

In [ ]:
for _, row in selected_reference_values.iterrows():
    if pd.isna(row.MIMIC_equivalent_name):
        if row.DPI_name == 'calcium corrige':
            continue
        else:
            raise ValueError(f'No MIMIC equivalent for {row.DPI_name}')
    equivalence_list = pd.DataFrame([row.MIMIC_equivalent_name, row.other_MIMIC_equivalents]).dropna()[0].tolist()
    aligned_lab_df.loc[
            aligned_lab_df['label'].isin(equivalence_list), 'label'] = row.DPI_name



## Create calcium corrige

-> search for albumin and calcium drawn at same time

In [ ]:
calcium_components_df = aligned_lab_df[aligned_lab_df.label.isin(['Calcium, Total', 'Albumin'])]
calcium_components_df['hadm_id_charttime'] = calcium_components_df['hadm_id'].astype(str) + '_' + calcium_components_df['charttime'].astype(str)
# calcium_components_df['label_hadm_id_charttime'] = calcium_components_df['hadm_id'].astype(str) + '_' + calcium_components_df['hadm_id_charttime'].astype(str)
calcium_components_df = calcium_components_df.drop_duplicates()[(calcium_components_df.drop_duplicates().duplicated(subset=['hadm_id_charttime'], keep=False))]

In [ ]:
corrected_calcium_df = calcium_components_df[calcium_components_df.label == 'Calcium, Total'].copy()
corrected_calcium_df['corrected_value'] = corrected_calcium_df['valuenum']
for index, row in tqdm(corrected_calcium_df.iterrows()):
    # Formula: adjusted [Ca](mmol/L) = total [Ca](mmol/L) + 0.02 (40 - [albumin](g/L))
    # Conversion factors:
    # - calcium (mg/dl -> mmol/L) : *0.2495
    # - albumin (g/dL -> g/L): *10
    simultaneous_albumin = calcium_components_df[(calcium_components_df.label == 'Albumin') & (calcium_components_df.charttime == row.charttime)]
    corrected_calcium_df.at[index,'corrected_value'] = row['corrected_value'] * 0.2495 + 0.02 * (40 - simultaneous_albumin.valuenum.values[0] * 10)
corrected_calcium_df['corrected_valueuom'] = 'mmol/l'

In [ ]:
corrected_calcium_df['label'] = 'calcium corrige'
corrected_calcium_df['value'] = corrected_calcium_df['corrected_value']
corrected_calcium_df['valuenum'] = corrected_calcium_df['corrected_value']
corrected_calcium_df['valueuom'] = corrected_calcium_df['corrected_valueuom']
corrected_calcium_df.drop(columns=['corrected_value', 'corrected_valueuom', 'hadm_id_charttime'], inplace=True)




In [ ]:
aligned_lab_df = pd.concat([aligned_lab_df, corrected_calcium_df])

## Restrict to selected variables

In [ ]:
selected_lab_labels = selected_reference_values['DPI_name'].tolist()
restricted_lab_df = aligned_lab_df.copy()
restricted_lab_df = restricted_lab_df[restricted_lab_df.label.isin(selected_lab_labels)]

In [ ]:
assert restricted_lab_df.label.unique().tolist().sort() == selected_lab_labels.sort()

## Convert Non numerical values
- ">" & "greater than" are replaced by value + 5% of value

In [ ]:
non_interpretable_non_numerical_values = ['NEG', 'TR', 'UNABLE TO REPORT', 'ERROR', 'UNABLE TO PERFORM']

In [ ]:
restricted_lab_df.loc[(restricted_lab_df.valuenum.isna()) & (restricted_lab_df.value.isin(non_interpretable_non_numerical_values)), 'value'] = np.nan

In [ ]:
restricted_lab_df.loc[restricted_lab_df['value'].str.contains(">", case=False, na=False), ['value', 'valuenum']] = \
    restricted_lab_df[restricted_lab_df['value'].str.contains(">", case=False, na=False)]\
        .apply(lambda row: float(row['value'].split('>')[1]) * 1.05 , axis=1)

restricted_lab_df.loc[restricted_lab_df['value'].str.contains("greater than ", case=False, na=False), ['value', 'valuenum']] = \
    restricted_lab_df[restricted_lab_df['value'].str.contains("greater than ", case=False, na=False)]\
        .apply(lambda row: float(row['value'].split('GREATER THAN ')[1]) * 1.05 , axis=1)

restricted_lab_df.loc[restricted_lab_df['value'].str.contains("IS HIGHEST MEASURED ", case=False, na=False), ['value', 'valuenum']] = \
    restricted_lab_df[restricted_lab_df['value'].str.contains("IS HIGHEST MEASURED ", case=False, na=False)]\
        .apply(lambda row: float(row['value'].split('IS HIGHEST MEASURED ')[0]) * 1.05 , axis=1)

In [ ]:

if verbose:
    print(f'Excluding {len(restricted_lab_df[(restricted_lab_df.valuenum.isna()) & (~restricted_lab_df.value.isna())])} values because non-numerical')

restricted_lab_df.drop(restricted_lab_df[(restricted_lab_df.valuenum.isna()) & (~restricted_lab_df.value.isna())].index, inplace=True)


In [ ]:
restricted_lab_df[(restricted_lab_df.valuenum.isna()) & (~restricted_lab_df.value.isna())]

## Convert units

In [ ]:
print('Converting units')
# Not perfect in terms of efficiency or conciseness, but no bottleneck + works + readable (-> leave like this for now)
for _, row in tqdm(selected_reference_values[selected_reference_values.unit_conversion_needed == 1].iterrows()):
    restricted_lab_df.valuenum = restricted_lab_df.apply(lambda X: X.valuenum * row['multiplicative_factor']
                                                            if X.label == row['DPI_name'] else X.valuenum, axis=1)
    restricted_lab_df.valueuom = restricted_lab_df.apply(lambda X: row['DPI_units']
                                                            if X.label == row['DPI_name'] else X.valueuom, axis=1)


Converting units for proteine C-reactive (some values are ine mg/dl and not mg/l)

In [ ]:
restricted_lab_df.loc[restricted_lab_df.valueuom == 'mg/dL', 'valuenum'] = restricted_lab_df[restricted_lab_df.valueuom == 'mg/dL']['valuenum'] * 10
restricted_lab_df.loc[restricted_lab_df.valueuom == 'mg/dL', 'valueuom'] = 'mg/l'

In [ ]:
restricted_lab_df.value = restricted_lab_df.valuenum.astype(str)

Replace units that are equivalent

In [ ]:
restricted_lab_df.loc[(restricted_lab_df['label'].isin(['chlore', 'sodium', 'potassium']))
                       & (restricted_lab_df['valueuom'] == 'mEq/L'), 'valueuom'] = 'mmol/l'
restricted_lab_df.loc[restricted_lab_df['valueuom'] == 'mmol/L', 'valueuom'] = 'mmol/l'
restricted_lab_df.loc[(restricted_lab_df['valueuom'] == 'IU/L'), 'valueuom'] = 'U/l'
restricted_lab_df.loc[(restricted_lab_df['valueuom'] == 'm/uL'), 'valueuom'] = 'T/l'
restricted_lab_df.loc[(restricted_lab_df['valueuom'] == 'K/uL'), 'valueuom'] = 'G/l'
restricted_lab_df.loc[(restricted_lab_df['valueuom'] == 'pg/mL'), 'valueuom'] = 'ng/l'

In [ ]:
for _, row in selected_reference_values.iterrows():
    if pd.isna(row.DPI_units):
        continue
    if row.DPI_units != restricted_lab_df[restricted_lab_df.label == row.DPI_name].valueuom.unique()[0]:
        raise ValueError(f'Units for {row.DPI_name} do not correspond', row.DPI_units, restricted_lab_df[restricted_lab_df.label == row.DPI_name].valueuom.unique()[0])

## Restrict to plausible value ranges

In [ ]:
plausible_restricted_lab_df = restricted_lab_df.copy()

In [ ]:
possible_value_ranges_file = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))),
                                              'preprocessing/possible_ranges_for_variables.xlsx')
possible_value_ranges = pd.read_excel(possible_value_ranges_file)

In [ ]:
plausible_restricted_lab_df['out_of_range'] = False
for variable in possible_value_ranges['variable_label'].dropna().unique():
    possible_value_ranges_for_variable = possible_value_ranges[possible_value_ranges['variable_label'] == variable]
    plausible_restricted_lab_df.loc[(plausible_restricted_lab_df['label'] == variable)
                             & (~plausible_restricted_lab_df['valuenum'].between(
                                         possible_value_ranges_for_variable['Min'].values[0],
                                         possible_value_ranges_for_variable['Max'].values[0])),
                                    'out_of_range'] = True


In [ ]:
n_observations_out_ouf_range = len(plausible_restricted_lab_df[(plausible_restricted_lab_df["out_of_range"]) & (~plausible_restricted_lab_df.valuenum.isna())])
if verbose:
    print(f'Excluding {n_observations_out_ouf_range} observations because out of range')

plausible_restricted_lab_df.loc[plausible_restricted_lab_df['out_of_range'] == True, ['value', 'valuenum']] = np.NAN

plausible_restricted_lab_df.dropna(subset=['valuenum'], inplace=True)

## Log descriptive stats

In [ ]:
# get mean number of values per dosage label patient admission id
median_observations_per_case_admission_id = \
    plausible_restricted_lab_df.groupby(['hadm_id', 'label'])['valuenum'].count().reset_index()
median_observations_per_case_admission_id_df = median_observations_per_case_admission_id.groupby('label').median()
median_observations_per_case_admission_id_df.rename(columns={'value': 'median_observations_per_case_admission_id'}, inplace=True)
descriptive_stats_df = plausible_restricted_lab_df.groupby('label')['valuenum'].describe()

In [ ]:
median_observations_per_case_admission_id_df

In [ ]:
descriptive_stats_df

## Testing full pipeline

In [ ]:
from mimic_preprocessing.lab_preprocessing.lab_preprocessing import preprocess_labs

preprocessed_labs = preprocess_labs(lab_df, log_dir='/Users/jk1/temp/mimic/prepro/test')

In [ ]:
preprocessed_labs[preprocessed_labs.valuenum.isna()]

Remaining issues:
- BNP does not have the same distribution in both datasets

In [ ]:
lab_df[lab_df.label == 'NTproBNP']